In [31]:
library(io)
source("../common-ffpe-snvf/R/eval.R")
source("../common-ffpe-snvf/R/plot.R")

### Testing MOBSNVF's performance before and after MicroSEC's Microhomology-Induced Chimeric Read (MICR) artifact filtering

#### EGAD00001004066 Dataset

In [19]:
microsec_res <- read.delim("EGAD00001004066/somatic_vcf/model-scores_truths/all-samples_microsec-scores_truths.tsv")
names(microsec_res)[names(microsec_res) == "score"] <- "microsec.score"
dim(microsec_res)
head(microsec_res)

[1] 15347     9

,sample_name,chrom,pos,ref,alt,msec_filter_all,microsec.score,snv,truth
,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<lgl>
1,FFPE-Liver-Tumoral_B00GXDN,chr1,901320,C,T,Artifact suspicious,0,chr1_901320_C_T,FALSE
2,FFPE-Liver-Tumoral_B00GXDN,chr1,916820,G,A,Artifact suspicious,0,chr1_916820_G_A,FALSE
3,FFPE-Liver-Tumoral_B00GXDN,chr1,1326795,G,A,,1,chr1_1326795_G_A,FALSE
4,FFPE-Liver-Tumoral_B00GXDN,chr1,2495318,C,T,Artifact suspicious,0,chr1_2495318_C_T,FALSE
5,FFPE-Liver-Tumoral_B00GXDN,chr1,2652919,G,A,,1,chr1_2652919_G_A,FALSE
6,FFPE-Liver-Tumoral_B00GXDN,chr1,2684246,G,A,,1,chr1_2684246_G_A,FALSE


In [ ]:
## Score is adjusted by adding 1 to keep it between 0 and 1. 
mobsnvf_res <- read.delim("EGAD00001004066/somatic_vcf/model-scores_truths/all-samples_mobsnvf-scores_truths.tsv")
names(mobsnvf_res)[names(mobsnvf_res) == "score"] <- "mobsnvf.score"
summary(mobsnvf_res$mobsnvf.score)
mobsnvf_res$mobsnvf.score <- mobsnvf_res$mobsnvf.score + 1
summary(mobsnvf_res$mobsnvf.score)
dim(mobsnvf_res)
head(mobsnvf_res)

      Min.    1st Qu.     Median       Mean    3rd Qu.       Max. 
-1.0000000 -0.0000857 -0.0000001 -0.1179469  0.0000000  0.0000000 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.0000  0.9999  1.0000  0.8821  1.0000  1.0000 

[1] 15347     9

,chrom,pos,ref,alt,FOBP,mobsnvf.score,snv,truth,sample_name
,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<lgl>,<chr>
1,chr1,901320,C,T,1.47301e-07,0.9999999,chr1_901320_C_T,FALSE,FFPE-Liver-Tumoral_B00GXDN
2,chr1,916820,G,A,2.66617e-07,0.9999997,chr1_916820_G_A,FALSE,FFPE-Liver-Tumoral_B00GXDN
3,chr1,1326795,G,A,1.10821e-08,1.0000000,chr1_1326795_G_A,FALSE,FFPE-Liver-Tumoral_B00GXDN
4,chr1,2495318,C,T,1.27492e-08,1.0000000,chr1_2495318_C_T,FALSE,FFPE-Liver-Tumoral_B00GXDN
5,chr1,2652919,G,A,9.43791e-07,0.9999991,chr1_2652919_G_A,FALSE,FFPE-Liver-Tumoral_B00GXDN
6,chr1,2684246,G,A,2.78648e-04,0.9997214,chr1_2684246_G_A,FALSE,FFPE-Liver-Tumoral_B00GXDN


In [33]:
combined_res <- merge(mobsnvf_res, microsec_res, by = c("sample_name", "chrom", "pos", "ref", "alt", "snv", "truth"))
head(combined_res)

,sample_name,chrom,pos,ref,alt,snv,truth,FOBP,mobsnvf.score,msec_filter_all,microsec.score
,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<lgl>,<dbl>,<dbl>,<chr>,<int>
1,FFPE-Colon-Tumoral_B00GXHK,chr1,100629871,C,T,chr1_100629871_C_T,FALSE,3.11028e-09,1.0000000,Artifact suspicious,0
2,FFPE-Colon-Tumoral_B00GXHK,chr1,100998233,G,A,chr1_100998233_G_A,FALSE,2.08840e-04,0.9997912,,1
3,FFPE-Colon-Tumoral_B00GXHK,chr1,10447745,C,T,chr1_10447745_C_T,FALSE,3.24098e-08,1.0000000,Artifact suspicious,0
4,FFPE-Colon-Tumoral_B00GXHK,chr1,10447751,G,A,chr1_10447751_G_A,FALSE,3.01818e-08,1.0000000,Artifact suspicious,0
5,FFPE-Colon-Tumoral_B00GXHK,chr1,104478793,C,T,chr1_104478793_C_T,FALSE,1.58642e-08,1.0000000,,1
6,FFPE-Colon-Tumoral_B00GXHK,chr1,104478796,C,T,chr1_104478796_C_T,FALSE,1.22083e-05,0.9999878,,1


In [21]:
# Mobsnvf performance evaluation
with(combined_res, evalmod(scores = mobsnvf.score, labels = truth, modnames = "mobsnvf"))


    === AUCs ===

     Model name Dataset ID Curve type       AUC
   1    mobsnvf          1        ROC 0.9243162
   2    mobsnvf          1        PRC 0.7270930


    === Input data ===

     Model name Dataset ID # of negatives # of positives
   1    mobsnvf          1          14213           1134


In [36]:
## Remove variant which are marked as artifacts by MicroSEC i.e score == 0. Not that is score is binary and not continuous
table(combined_res$microsec.score)
filtered_res <- combined_res[combined_res$microsec.score == 1, ]
dim(filtered_res)
head(filtered_res)


   0    1 
9385 5962 

[1] 5962   11

,sample_name,chrom,pos,ref,alt,snv,truth,FOBP,mobsnvf.score,msec_filter_all,microsec.score
,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<lgl>,<dbl>,<dbl>,<chr>,<int>
2,FFPE-Colon-Tumoral_B00GXHK,chr1,100998233,G,A,chr1_100998233_G_A,FALSE,2.08840e-04,0.9997912,,1
5,FFPE-Colon-Tumoral_B00GXHK,chr1,104478793,C,T,chr1_104478793_C_T,FALSE,1.58642e-08,1.0000000,,1
6,FFPE-Colon-Tumoral_B00GXHK,chr1,104478796,C,T,chr1_104478796_C_T,FALSE,1.22083e-05,0.9999878,,1
8,FFPE-Colon-Tumoral_B00GXHK,chr1,108692428,G,A,chr1_108692428_G_A,FALSE,3.69858e-03,0.9963014,,1
12,FFPE-Colon-Tumoral_B00GXHK,chr1,113980694,C,T,chr1_113980694_C_T,FALSE,1.35675e-07,0.9999999,,1
13,FFPE-Colon-Tumoral_B00GXHK,chr1,119514034,G,A,chr1_119514034_G_A,TRUE,0.00000e+00,1.0000000,,1


In [39]:
# Evaluation after filtering
with(filtered_res, evalmod(scores = mobsnvf.score, labels = truth, modnames = "mobsnvf"))


    === AUCs ===

     Model name Dataset ID Curve type       AUC
   1    mobsnvf          1        ROC 0.9195341
   2    mobsnvf          1        PRC 0.7894955


    === Input data ===

     Model name Dataset ID # of negatives # of positives
   1    mobsnvf          1           4914           1048


In [45]:
sprintf("Change in AUROC: %.03f", (0.9195341 - 0.9243162))
sprintf("Change in AUPRC: %.03f", (0.7894955 - 0.7270930))

[1] "Change in AUROC: -0.005"

[1] "Change in AUPRC: 0.062"

In [56]:
with(combined_res[!((combined_res$microsec.score == 0) & (!combined_res$truth)) , ], evalmod(scores = mobsnvf.score, labels = truth, modnames = "mobsnvf"))


    === AUCs ===

     Model name Dataset ID Curve type       AUC
   1    mobsnvf          1        ROC 0.9113118
   2    mobsnvf          1        PRC 0.7896328


    === Input data ===

     Model name Dataset ID # of negatives # of positives
   1    mobsnvf          1           4914           1134
